In [1]:
import errno
import glob
import json
import os
import re
import shutil
from types import SimpleNamespace
import cv2
import torch
import warnings
from lightning_objects import LightningModel
warnings.filterwarnings('ignore')
from config import Configuration
import pandas as pd
from utils import stratify_split, make_holdout_df, set_seeds
from train_manager import TrainManager

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def main(experiment_name: str, debug, resume=False,
         finetune=False, freeze_bn=True, freeze_feature_extractor=False):

    experiment_dir = os.path.abspath(f'trained-models/{experiment_name}')
    print('Experiment directory', experiment_dir)

    try:
        # -------- SETUP --------
        checkpoint_params = None
        finetune_model_fnames = None
        folds_df, holdout_df = None, None

        if not resume and not finetune: # totally new experiment
            make_experiment_directory(experiment_dir)
            config = Configuration()
            config.debug = debug
            set_seeds(config.seed)

            # -------- LOAD DATA FROM TRAIN FILE --------
            data_df = pd.read_csv(config.data_dir + '/train.csv', engine='python')
            data_df, holdout_df = make_holdout_df(data_df, seed=config.seed)
            folds_df = stratify_split(data_df, config.fold_num, config.seed, config.target_col)

            # -------- SAVE FILES (for experiment state) --------
            folds_df.to_csv(experiment_dir + '/folds.csv', index=False)
            # save holdout to a csv file for final inference (so we don't run inference on training examples)
            holdout_df.to_csv(experiment_dir + '/holdout.csv', index=False)
            with open(experiment_dir + '/experiment_config.json', 'w') as f:
                json.dump(config.__dict__, f)
        elif resume or finetune:
            # LOAD DATA FROM SAVED FILES
            with open(experiment_dir + '/experiment_config.json', 'r') as f:
                config = json.load(f, object_hook=lambda d: SimpleNamespace(**d))
                set_seeds(config.seed)
                config.debug = debug

            folds_df = pd.read_csv(experiment_dir + '/folds.csv', engine='python')
            holdout_df = pd.read_csv(experiment_dir + '/holdout.csv', engine='python')

            if finetune and not resume:
                print('finetuning...')
                # verify there are checkpoints to fine tune
                finetune_model_fnames = glob.glob(experiment_dir + '/*fold*.ckpt')
                assert len(finetune_model_fnames) > 0
                finetune_model_fnames.sort()

                # make new directory for tuning experiment with files from training run 1
                make_experiment_directory(experiment_dir + '_tune')
                for f in os.listdir(experiment_dir):
                    print(f"copying {f} to {experiment_dir + '_tune'}")
                    shutil.copy2(experiment_dir + '/' + f, experiment_dir + '_tune')
                experiment_dir += '_tune'
                experiment_name += '_tune'
            else:
                print('resuming from last checkpoint...')
                checkpoint_params = get_checkpoint_params(experiment_dir, resume, config.model_arch)

        assert holdout_df is not None, 'holdout_df is None'
        assert folds_df is not None, 'folds_df is None'

        # cv2 multithreading seems to go into deadlock with PyTorch data loaders
        if config.num_workers > 0:
            cv2.setNumThreads(0)

        trainer = TrainManager(experiment_name=experiment_name, experiment_dir=experiment_dir,
                               folds_df=folds_df, holdout_df=holdout_df,
                               checkpoint_params=checkpoint_params, config=config,
                               finetune=finetune, freeze_bn=freeze_bn,
                               freeze_feature_extractor=freeze_feature_extractor,
                               finetune_model_fnames=finetune_model_fnames)
        trainer.run()
    finally:
        torch.cuda.empty_cache()

def make_experiment_directory(name):
    try:
        os.makedirs(name)
    except FileExistsError as e:
        print('Experiment already exists. Be sure to resume training appropriately or start a new experiment.')
        if e.errno == errno.EEXIST: raise


def get_checkpoint_params(basename, resume, model_arch):
    checkpoint_params = None
    if resume:
        checkpoint_params = {}
        model_filenames = glob.glob(basename + '/*fold*.ckpt')
        model_filenames.sort()
        trained_folds = [re.findall(r'fold\d+', f)[0][len('fold'):] for f in model_filenames]
        most_recent_fold = int(max(trained_folds)) if len(trained_folds) > 0 else 0

        checkpoint_params['restart_from'] = most_recent_fold
        checkpoint_params['checkpoint_file_path'] = model_filenames[-1]

    return checkpoint_params

In [ ]:
if __name__ == '__main__':
    try:
        debug = False
        print('Running in debug mode:', debug)
        main(experiment_name='seresnet50_sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0', debug=debug,
             resume=False, finetune=False, freeze_bn=True, freeze_feature_extractor=False)
    except KeyboardInterrupt:
        pass

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Running in debug mode: False
Experiment directory /opt/favordata/AI/Felix/kaggle-cassava/trained-models/seresnet50_sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0
folds_df len 18187, holdout_df len 3210
Training fold 1
Class sample counts [ 758 1471 1622 8933 1765]
After class sample counts [2274 2942 3730 8933 4765]
conv1.weight True
bn1.weight False
bn1.bias False
layer1.0.conv1.weight True
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight True
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.0.conv3.weight True
layer1.0.bn3.weight False
layer1.0.bn3.bias False
layer1.0.se.fc1.weight True
layer1.0.se.fc1.bias True
layer1.0.se.fc2.weight True
layer1.0.se.fc2.bias True
layer1.0.downsample.0.weight True
layer1.0.downsample.1.weight False
layer1.0.downsample.1.bias False
layer1.1.conv1.weight True
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight True
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer1.1.conv3.weight True
laye


  | Name           | Type           | Params
--------------------------------------------------
0 | valid_accuracy | Accuracy       | 0     
1 | test_accuracy  | Accuracy       | 0     
2 | criterion      | BiTemperedLoss | 0     
3 | model          | ResNet         | 26.0 M
--------------------------------------------------
26.0 M    Trainable params
53.1 K    Non-trainable params
26.0 M    Total params
Finding best initial lr: 100%|██████████| 100/100 [01:16<00:00,  1.29it/s]Restored states from the checkpoint file at /opt/favordata/AI/Felix/kaggle-cassava/trained-models/seresnet50_sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0/lr_find_temp_model.ckpt
Learning rate set to 0.2290867652767775

  | Name           | Type           | Params
--------------------------------------------------
0 | valid_accuracy | Accuracy       | 0     
1 | test_accuracy  | Accuracy       | 0     
2 | criterion      | BiTemperedLoss | 0     
3 | model          | ResNet         | 26.0 M
--------------

Epoch 1:  94%|█████████▍| 910/967 [03:00<00:20,  2.83it/s, loss=0.2, v_num=0, val_loss=0.49, val_acc=0.195, train_loss=0.186]      
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 57/57 [00:16<00:00,  1.77it/s]

Epoch 1, global step 228: val_loss reached 0.14008 (best 0.14008), saving model to "/opt/favordata/AI/Felix/kaggle-cassava/trained-models/seresnet50_sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0/seresnet50_bitempered_smooth=0.05_val_loss=0.140_val_acc=0.840_fold1.ckpt" as top 1


...

Epoch 19, step 4332: val_loss was not in top 1


Epoch 19: 100%|██████████| 967/967 [03:14<00:00,  2.93it/s, loss=0.0925, v_num=0, val_loss=0.112, val_acc=0.882, train_loss=0.0388]
                                                           

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold 2
Class sample counts [ 759 1471 1622 8934 1764]
After class sample counts [2277 2942 3730 8934 4762]
conv1.weight True
bn1.weight False
bn1.bias False
layer1.0.conv1.weight True
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight True
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.0.conv3.weight True
layer1.0.bn3.weight False
layer1.0.bn3.bias False
layer1.0.se.fc1.weight True
layer1.0.se.fc1.bias True
layer1.0.se.fc2.weight True
layer1.0.se.fc2.bias True
layer1.0.downsample.0.weight True
layer1.0.downsample.1.weight False
layer1.0.downsample.1.bias False
layer1.1.conv1.weight True
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight True
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer1.1.conv3.weight True
layer1.1.bn3.weight False
layer1.1.bn3.bias False
layer1.1.se.fc1.weight True
layer1.1.se.fc1.bias True
layer1.1.se.fc2.weight True
layer1.1.se.fc2.bias True
layer1.2.conv1.weight True
layer1.2.bn1.weight False
l


  | Name           | Type           | Params
--------------------------------------------------
0 | valid_accuracy | Accuracy       | 0     
1 | test_accuracy  | Accuracy       | 0     
2 | criterion      | BiTemperedLoss | 0     
3 | model          | ResNet         | 26.0 M
--------------------------------------------------
26.0 M    Trainable params
53.1 K    Non-trainable params
26.0 M    Total params
Finding best initial lr:   2%|▏         | 2/100 [00:00<00:52,  1.86it/s]